# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][HW04] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [116]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from collections import Counter 
from catboost import Pool, CatBoostClassifier

In [3]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

class DecisionTreeNode:
    def __init__(self, split_dim, split_value, left, right):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right

class DecisionTreeLeaf:
    def __init__(self, probs, size):
        self.probs = probs
        self.size = size
        self.y = max(probs, key=probs.get)`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [28]:
def bagging(X, y):
    Xb = np.zeros(X.shape)
    yb = np.zeros(y.shape, dtype=y.dtype)
    ind = set(range(X.shape[0]))
    for i in range(X.shape[0]):
        j = np.random.randint(X.shape[0])
        Xb[i] = X[j]
        yb[i] = y[j]
        ind.discard(j)
    ind = list(ind)
    Xoob = np.zeros(shape=(len(ind),X.shape[1]))
    yoob = np.zeros(len(ind), dtype=y.dtype)
    for i in range(len(ind)):
        Xoob[i] = X[ind[i]]
        yoob[i] = y[ind[i]]
    return Xb, yb, Xoob, yoob

In [110]:
class DecisionTreeNode:
    def __init__(self, split_dim, split_value, left, right):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right

class DecisionTreeLeaf:
    def __init__(self, probs, size):
        self.probs = probs
        self.size = size
        self.y = max(probs, key=probs.get)

class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        
        if max_features == "auto":
            self.max_features = int(np.sqrt(X.shape[1]))
        else:
            self.max_features = max_features
        if criterion == "gini":
            self.criterion = gini
        elif criterion == "entropy":
            self.criterion = entropy
        else:
            raise ValueError(f"wrong criterion {criterion}")
            
        self.X, self.y, self.Xoob, self.yoob = bagging(X, y)
        self.fit(self.X, self.y)
            
    def make_node(self, ind, X, y, depth):
        def make_leaf():
            uni = np.unique(y)
            probs = dict(zip(uni, np.zeros(len(uni))))
            cur_uni, cnt = np.unique(y[ind], return_counts=True)
            probs2 = dict(zip(cur_uni, cnt/len(ind)))
            probs.update(probs2)
            return DecisionTreeLeaf(probs, len(ind))
        
        if (self.max_depth is not None and depth >= self.max_depth) or len(ind) < 2*self.min_samples_leaf or len(np.unique(y[ind])) == 1:
            return make_leaf()
        else:
            best_f = None
            best_s = 0.5
            best_left = None
            best_right = None
            best_gain = None

            for k in np.random.choice(range(X.shape[1]), self.max_features, replace=False):
                cur_left = ind[X[ind, k] < 0.5]
                cur_right = ind[X[ind, k] > 0.5]
                if (len(cur_left) >= self.min_samples_leaf and len(cur_right)>= self.min_samples_leaf) or len(cur_right) == 0:
                    cur_gain = gain(y[cur_left], y[cur_right], self.criterion)
                    if best_gain is None or cur_gain > best_gain:
                        best_f = k
                        best_left = cur_left
                        best_right = cur_right
                        best_gain = cur_gain
            if best_right is None or len(best_right) == 0:
                return make_leaf()
            else:
                right_son = self.make_node(best_right, X, y, depth+1)
                left_son = self.make_node(best_left, X, y, depth+1)
                cur_node = DecisionTreeNode(best_f, best_s, left_son, right_son)
                return cur_node
                
            
    def fit(self, X, y):
        self.root = self.make_node(np.arange(X.shape[0]), X, y, 0)
    
    def query(self, x):
        cur_node = self.root
        while not isinstance(cur_node, DecisionTreeLeaf):
            if x[cur_node.split_dim]  <= cur_node.split_value:
                cur_node = cur_node.left
            else:
                cur_node = cur_node.right
        return cur_node.probs

    def predict_proba(self, X):
        ans = []
        for i in range(X.shape[0]):
            ans.append(self.query(X[i]))
        return ans
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return [max(p.keys(), key=lambda k: p[k]) for p in proba]
    
    def error(self, X_test, y_test):
        y_pred = self.predict(X_test)
        err = len([1 for i in zip(y_pred, y_test) if i[0] != i[1]])
        return err/len(y_test)
    
    def feature_importance(self):
        err = self.error(self.Xoob, self.yoob)
        res = np.zeros(self.X.shape[1])
        for i in range(self.X.shape[1]):
            sX = self.Xoob.copy()
            np.random.shuffle(sX[:,i])
            res[i] = self.error(sX, self.yoob) - err
        return res

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [65]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.forest = [None]*n_estimators
    
    def fit(self, X, y):
        self.dtyp = y.dtype
        self.features = X.shape[1]
        for i in range(len(self.forest)):
            self.forest[i] = DecisionTree(X, y, self.criterion, self.max_depth, self.min_samples_leaf, self.max_features)
    
    
    def predict(self, X):
        res = np.zeros(shape=(X.shape[0], len(self.forest)), dtype=self.dtyp)
        for i in range(len(self.forest)):
            res[:,i] = self.forest[i].predict(X)
        cls = []
        for i in range(X.shape[0]):
            cls.append(Counter(res[i]).most_common(1)[0][0])
        return np.array(cls)

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [17]:
def feature_importance(rfc):
    res = np.zeros(rfc.features)
    for tree in rfc.forest:
        res += tree.feature_importance()
    return res / len(rfc.forest)

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате должна получиться точность `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [55]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [-1.23328780e-03 -3.46393375e-04  1.56848928e-01  1.66211011e-01
  3.21926431e-01  2.83577569e-04]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.

Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.

Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [56]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [108]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [109]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.7238335435056746
Most important features:
1. ovsyanochan
2. 4ch
3. styd.pozor
4. rhymes
5. dayvinchik
6. rapnewrap
7. mudakoff
8. iwantyou
9. pixel_stickers
10. memeboizz
11. pravdashowtop
12. leprum
13. ne1party
14. bot_maxim
15. reflexia_our_feelings
16. borsch
17. soverwenstvo.decora
18. thesmolny
19. pozor
20. onlyorly


#### Пол

In [67]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8511979823455234
Most important features:
1. 40kg
2. 4ch
3. modnailru
4. zerofat
5. girlmeme
6. mudakoff
7. i_d_t
8. femalemem
9. cook_good
10. 9o_6o_9o
11. sh.cook
12. rapnewrap
13. fuck_humor
14. be.beauty
15. be.women
16. psy.people
17. igm
18. thesmolny
19. reflexia_our_feelings
20. club52205838


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.

Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [140]:
model = CatBoostClassifier(loss_function='MultiClass')
X, y = synthetic_dataset(1000)
train_dataset = Pool(data=X,
                     label=y,
                     cat_features=None)
model.fit(train_dataset, silent=True)

In [143]:
X, y = synthetic_dataset(1000)

print("Accuracy:", np.mean(model.predict(train_dataset).reshape(y.shape) == y))
print("Importance:", model.get_feature_importance())

Accuracy: 1.0
Importance: [1.24341533e-02 1.97110774e-03 2.78396701e+01 2.78290121e+01
 4.43092689e+01 7.64369948e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.

Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [144]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [156]:
model = CatBoostClassifier(loss_function='MultiClass')
train_dataset = Pool(data=X_train,
                     label=y_age_train,
                     cat_features=None)
test_dataset = Pool(data=X_test,
                    label=y_age_test,
                    cat_features=None)

model.fit(train_dataset, eval_set=(X_eval, y_age_eval), silent=True)

print("Accuracy:", np.mean(model.predict(test_dataset).reshape(y_age_test.shape) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.7061790668348046
Most important features:
1. ovsyanochan
2. styd.pozor
3. mudakoff
4. 4ch
5. dayvinchik
6. rhymes
7. leprum
8. xfilm
9. rapnewrap
10. iwantyou
11. in.humour
12. tumblr_vacuum
13. fuck_humor
14. bot_maxim
15. dzenpub
16. i_des
17. pixel_stickers
18. bestad
19. thesmolny
20. bon


#### Пол

In [152]:
model = CatBoostClassifier(loss_function='MultiClass')
train_dataset = Pool(data=X_train,
                     label=y_sex_train,
                     cat_features=None)
test_dataset = Pool(data=X_test,
                    label=y_sex_test,
                    cat_features=None)

model.fit(train_dataset, eval_set=(X_eval, y_sex_eval), silent=True)

print("Accuracy:", np.mean(model.predict(test_dataset).reshape(y_sex_test.shape) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8549810844892812
Most important features:
1. 40kg
2. girlmeme
3. modnailru
4. mudakoff
5. thesmolny
6. femalemem
7. igm
8. 9o_6o_9o
9. i_d_t
10. be.beauty
11. academyofman
12. reflexia_our_feelings
13. rapnewrap
14. bon
15. zerofat
16. woman.blog
17. sh.cook
18. cook_good
19. bot_maxim
20. science_technology
